# 데이터 로드

In [1]:
song_meta = pd.read_json('./raw_data/song_meta.json')
train = pd.read_json('./raw_data/train.json')
val = pd.read_json('./raw_data/val.json')
genre = pd.read_json('./raw_data/genre_gn_all.json', typ = 'seriese')

## id별 장르 구분

In [2]:
## id별 장르 구분

song_meta[['id','song_gn_dtl_gnr_basket','song_gn_gnr_basket']]

song_meta['total_gn'] = song_meta['song_gn_dtl_gnr_basket'] + song_meta['song_gn_gnr_basket']

song_meta[['id','total_gn']]

df = song_meta[['id','total_gn']].explode('total_gn')
# df1.explode('song_gn_dtl_gnr_basket')
df

,id,total_gn
0,0,GN0901
0,0,GN0900
1,1,GN1601
1,1,GN1606
1,1,GN1600
...,...,...
707987,707987,GN1801
707987,707987,GN1800
707988,707988,GN0601
707988,707988,GN0604


In [3]:
df1 = pd.get_dummies(df['total_gn']).max(level=0).reset_index()

In [4]:
df1.rename(columns={'index': 'id'}, inplace=True)

In [5]:
df1

,id,GN0100,GN0101,GN0102,GN0103,GN0104,GN0105,GN0200,GN0201,GN0202,...,GN2802,GN2803,GN2804,GN2805,GN2806,GN2900,GN2901,GN2902,GN2903,GN9000
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707984,707984,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
707985,707985,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
707986,707986,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
707987,707987,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 연, 월

In [6]:
song_meta['issue_year'] = song_meta['issue_date'].apply(lambda x : str(x)[:4] if len(str(x)) != 1 else '0' )

In [7]:
song_meta['issue_month'] = song_meta['issue_date'].apply(lambda x :  str(x)[4:6] if len(str(x)) != 1 else '0')

In [8]:
song_meta[['id','issue_year','issue_month']]

,id,issue_year,issue_month
0,0,2014,05
1,1,2008,04
2,2,2018,05
3,3,2015,10
4,4,2011,08
...,...,...,...
707984,707984,1999,12
707985,707985,1986,00
707986,707986,2016,01
707987,707987,2013,12


## df 합체

In [49]:
df = df1.merge(song_meta[['id','issue_year','issue_month']],on='id')

In [10]:
song_meta.shape

(707989, 12)

In [11]:
df

,id,GN0100,GN0101,GN0102,GN0103,GN0104,GN0105,GN0200,GN0201,GN0202,...,GN2804,GN2805,GN2806,GN2900,GN2901,GN2902,GN2903,GN9000,issue_year,issue_month
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2014,05
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2008,04
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2018,05
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2015,10
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2011,08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707984,707984,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1999,12
707985,707985,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1986,00
707986,707986,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,2016,01
707987,707987,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2013,12


## playlist like

In [12]:
# 함수 사용 안함
def playlist_like_cnt(x):
    ls = []
    for song in x['songs']:
        ls.append({'song':song,'like_cnt':x['like_cnt']})       
    return ls

In [13]:
ls = []
for i in train[['songs','like_cnt']].index:
    songs = train.get_value(i,'songs')
    like_cnt = train.get_value(i,'like_cnt')
    for song in songs:
        ls.append({'id':song,'like_cnt': like_cnt})

In [14]:
result_df = pd.DataFrame(ls)

In [15]:
playlist_like_df = result_df.groupby('id').agg(sum)

In [16]:
playlist_like_df

,like_cnt
id,
0,29
3,1686
4,39
5,391
6,24
...,...
707984,2
707985,11
707986,1123


## df 합체

In [50]:
df = df.merge(playlist_like_df, on='id',how='outer')

In [18]:
df

,id,GN0100,GN0101,GN0102,GN0103,GN0104,GN0105,GN0200,GN0201,GN0202,...,GN2805,GN2806,GN2900,GN2901,GN2902,GN2903,GN9000,issue_year,issue_month,like_cnt
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2014,05,29.0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2008,04,NaN
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2018,05,NaN
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2015,10,1686.0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2011,08,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707984,707984,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1999,12,2.0
707985,707985,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1986,00,11.0
707986,707986,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,2016,01,1123.0
707987,707987,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2013,12,4.0


# playlist에 담겨진 횟수

In [19]:
ls = []
for i in train[['id','songs']].index:
    songs = train.get_value(i,'songs')
    playlist_id = train.get_value(i,'id')
    for song in songs:
        ls.append({'id':song,'population': playlist_id})

In [20]:
playlits_id = pd.DataFrame(ls)

In [21]:
population_df = playlits_id.groupby('id').count()

## df 합체

In [51]:
df = df.merge(population_df, on='id',how='outer')

In [23]:
## population이 nan 이면
## playlist에도 안들어가 있고
## 따라서 like_cnt도 없다!
df[df['population'].isnull()]

,id,GN0100,GN0101,GN0102,GN0103,GN0104,GN0105,GN0200,GN0201,GN0202,...,GN2806,GN2900,GN2901,GN2902,GN2903,GN9000,issue_year,issue_month,like_cnt,population
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2008,04,NaN,NaN
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2018,05,NaN,NaN
8,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2007,06,NaN,NaN
13,13,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2016,11,NaN,NaN
15,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2014,09,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707962,707962,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2014,08,NaN,NaN
707968,707968,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2013,05,NaN,NaN
707971,707971,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1996,00,NaN,NaN
707973,707973,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1997,00,NaN,NaN


# Tag score

### tag 전체에서 몇번 들어 갔는지

In [24]:
ls = []
for i in train[['tags','id']].index:
    tags = train.get_value(i,'tags')
    playlist_id = train.get_value(i,'id')
    for tag in tags:
        ls.append({'tag_id':tag,'count': playlist_id})

In [25]:
tag_df = pd.DataFrame(ls)

In [26]:
tag_count_df = tag_df.groupby('tag_id').count().reset_index()

In [27]:
tag_count_df

,tag_id,count
0,,1
1,00,4
2,007,3
3,007시리즈,2
4,00s,3
...,...,...
29155,힙해,5
29156,힙힙힙,2
29157,힛뎀포크,1
29158,힛뎀폭,1


## 노래별로 어떤 tag가 붙어있는지

In [28]:
ls = []
for i in train[['tags','songs']].index:
    tags = train.get_value(i,'tags')
    songs = train.get_value(i,'songs')
    for song in songs:
        ls.append({'id':song,'tags': tags})

In [29]:
song_tag = pd.DataFrame(ls)

In [30]:
song_tag_df = song_tag.groupby('id').agg(sum).reset_index()

In [31]:
song_tag_df

,id,tags
0,0,"[비오는날, 드라이브, 비오는날, 추억, 회상]"
1,3,"[1, 감성, 듣기좋은, 드라이브, Pop, 여행, 기분전환, 팝, 신나는, 클럽,..."
2,4,"[뉴에이지, 뉴에이지]"
3,5,"[비오는날, 재즈, 락, 발라드, 휴식, 리메이크, 재즈, 힐링송, 힐링, 기분전환..."
4,6,"[클래식, 산책, 조깅]"
...,...,...
615137,707984,[월드뮤직]
615138,707985,"[Pop, 해외차트, 시대별차트, 신디로퍼, Pop, 1980, 라이벌, 록]"
615139,707986,"[힙합, 랩, 발라드, 감성, 리메이크곡, 카페, 잔잔한, 추억, 위로, 힐링, 사..."
615140,707987,"[집중, 노동요, 새벽, 오후, 자장가, 공부]"


## 노래별로 붙어있는 tag의 count를 구해서 다 더 해주자

In [32]:
song_tag_df.iloc[0]['tags']

['비오는날', '드라이브', '비오는날', '추억', '회상']

In [33]:
from tqdm import tqdm
ls = []
for i in tqdm(song_tag_df[['id','tags']].index):
    id_ = song_tag_df.get_value(i, 'id')
    tags = song_tag_df.get_value(i, 'tags')
    ls.append({
        "id": id_,
        "count": tag_count_df[tag_count_df['tag_id'].isin(tags)]['count'].values.sum()
    })

100%|██████████| 615142/615142 [31:58<00:00, 320.64it/s]


In [42]:
tag_score = pd.DataFrame(ls)

In [43]:
tag_score.shape

(615142, 2)

In [63]:
tag_score = tag_score.rename({'count':'tag_score'},axis='columns')

In [64]:
tag_score

,id,tag_score
0,0,24176
1,3,79205
2,4,3493
3,5,59069
4,6,4982
...,...,...
615137,707984,243
615138,707985,4323
615139,707986,162809
615140,707987,12775


## df 합체

In [65]:
final_df = df.merge(tag_score, on='id',how='outer')

In [67]:
song_meta.shape

(707989, 12)

In [70]:
song_meta_KNN_df = song_meta.merge(final_df, on='id')

## df pc 저장 (용량 큽니다)

In [ ]:
final_df.to_json('./df_for_KNN.json')

In [72]:
song_meta_KNN_df.to_json('./df_for_KNN_and_song_meta.json')

## KNN 테스트 진행해보면 됩니다

In [79]:
final_df

,id,GN0100,GN0101,GN0102,GN0103,GN0104,GN0105,GN0200,GN0201,GN0202,...,GN2900,GN2901,GN2902,GN2903,GN9000,issue_year,issue_month,like_cnt,population,tag_score
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2014,05,29.0,2.0,24176.0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2008,04,NaN,NaN,NaN
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2018,05,NaN,NaN,NaN
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2015,10,1686.0,8.0,79205.0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2011,08,39.0,2.0,3493.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707984,707984,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1999,12,2.0,1.0,243.0
707985,707985,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1986,00,11.0,4.0,4323.0
707986,707986,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,2016,01,1123.0,11.0,162809.0
707987,707987,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2013,12,4.0,1.0,12775.0


In [81]:
final_df['tag_score'].describe()

count    615142.000000
mean      36540.735828
std       50220.117203
min           1.000000
25%        3493.000000
50%       15728.000000
75%       48018.000000
max      370989.000000
Name: tag_score, dtype: float64

In [80]:
final_df['population'].describe()

count    615142.000000
mean          8.592928
std          39.481912
min           1.000000
25%           1.000000
50%           2.000000
75%           4.000000
max        2175.000000
Name: population, dtype: float64

In [78]:
final_df['like_cnt'].describe()

count    615142.000000
mean       1508.641172
std        6652.869225
min           0.000000
25%          13.000000
50%          76.000000
75%         563.000000
max      326947.000000
Name: like_cnt, dtype: float64